# Training a Street Situation Detection Network
This project deals with the problem of detecting street situation images (images which are taken on the outside). The used dataset for training the network, was self-created. In order to get a copy of the dataset, contact s0558366@htw-berlin.de

## Requirements
### Imports

In [ ]:
import os
import pathlib

import tensorflow as tf
from tensorflow import keras

# Helper libraries
import matplotlib.pyplot as plt

print('TensorFlow version: {}'.format(tf.__version__))

## Training Configuration

In [ ]:
BATCH_SIZE = 32
EPOCHS = 1

IMG_HEIGHT = 299
IMG_WIDTH = 299

## Dataset
### Training dataset

The training dataset is a collection of street situations. The positive examples for a street image, were randomly selected from following datasets:

- [BDD100K Dataset](https://bair.berkeley.edu/blog/2018/05/30/bdd/)
- [Mapillary Vistas Dataset](https://www.mapillary.com/dataset/vistas?pKey=q0GhQpk20wJm1ba1mfwJmw&lat=20&lng=0&z=1.5)
- [Cityscapes Dataset](https://www.cityscapes-dataset.com/)

The negative examples were randomly selected from the [ImageNet](http://image-net.org/) validation dataset. Overall there are 12.000 images, split into a into a `train` (10.000 images) and `validation` (2000) images dataset. In each dataset the percentage of positive and negative example is 50%.#

In [ ]:
street_situation_dataset_path = "./street-situation-dataset"

image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

In [ ]:
train_data_dir = os.path.join(street_situation_dataset_path, "train")
train_data_dir = pathlib.Path(train_data_dir)

train_data_gen = image_generator.flow_from_directory(directory=str(train_data_dir),
                                                     batch_size=BATCH_SIZE,
                                                     shuffle=True,
                                                     target_size=(IMG_HEIGHT, IMG_WIDTH))

In [ ]:
val_data_dir = os.path.join(street_situation_dataset_path, "validation")
val_data_dir = pathlib.Path(val_data_dir)

val_data_gen = image_generator.flow_from_directory(directory=str(val_data_dir),
                                                     batch_size=BATCH_SIZE,
                                                     shuffle=True,
                                                     target_size=(IMG_HEIGHT, IMG_WIDTH))

### Evaluation Images


In [ ]:
test_data_dir = "./evaluation-images"
test_data_dir = pathlib.Path(test_data_dir)

test_data_gen = image_generator.flow_from_directory(directory=str(test_data_dir),
                                                    batch_size=BATCH_SIZE,
                                                    shuffle=True,
                                                    target_size=(IMG_HEIGHT, IMG_WIDTH))

### Visualization

In [ ]:
def show_batch(image_batch, label_batch):
    plt.figure(figsize=(15,15))
    for n in range(20):
        ax = plt.subplot(5,5,n+1)
        plt.imshow(image_batch[n])
        plt.axis('off')
image_batch, label_batch = next(train_data_gen)
show_batch(image_batch, label_batch)

## Street Situation Detection Network

### Building the model

In [ ]:
def create_model():
    base_model = keras.applications.ResNet50V2(weights="imagenet", include_top=False)
    for layer in base_model.layers:
        layer.trainable = False
    
    model = base_model.output
    model = keras.layers.GlobalAveragePooling2D()(model)
    model = keras.layers.Dense(1024, activation='relu')(model)
    
    predictions = keras.layers.Dense(2, activation='softmax')(model)
    return keras.models.Model(inputs=base_model.input, outputs=predictions)

In [ ]:
model = create_model()
model.summary()

In [ ]:
model.compile(optimizer='adam', 
              loss='categorical_crossentropy',
              metrics=['accuracy'])

### Callbacks

In [ ]:
log_dir = os.path.join('logs')
os.makedirs("logs", exist_ok=True)

checkpoint_dir= os.path.join(logdir, 'model_{epoch}')

In [ ]:
tensorboard_cbk = keras.callbacks.TensorBoard(log_dir=log_dir,
                                              write_images=True,
                                              histogram_freq=0,  # How often to log histogram visualizations
                                              embeddings_freq=0,  # How often to log embedding visualizations
                                              update_freq='epoch')

In [ ]:
checkpoint_cbk = keras.callbacks.ModelCheckpoint(filepath=checkpoint_dir,
                                                 # Path where to save the model
                                                 # The two parameters below mean that we will overwrite
                                                 # the current checkpoint if and only if
                                                 # the `val_loss` score has improved.
                                                 save_best_only=True,
                                                 monitor='val_loss',
                                                 verbose=1)

In [ ]:
callbacks = [tensorboard_cbk, checkpoint_cbk,]

### Train the model

In [ ]:
model.fit(train_data_gen, epochs=EPOCHS, validation_data=val_data_gen,  callbacks=callbacks)

### Evaluation

In [ ]:
test_loss, test_acc = model.evaluate(test_data_dir)
print('\nTest accuracy: {}'.format(test_acc))